In [ ]:
import numpy as np
import holoviews as hv

from bokeh.models import HoverTool
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore

np.random.seed(10)
hv.extension('bokeh')

This example demonstrates how to display multiple timeseries on a same plot leveraging `subcoordinate_y` and how to create and link a minimap to this plot using the `RangeToolLink`. More specifically, the example generates fake [Electroencephalography](https://en.wikipedia.org/wiki/Electroencephalography) (EEG) timeseries data, plots the timeseries together, computes the [z score](https://en.wikipedia.org/wiki/Standard_score) of each timeseries and creates an image of that, that serves as a minimap linked to the main plot for easier exploration.

## EEG data and plot

We start by creating the timeseries data that will be plotted together.

In [ ]:
n_channels = 10
n_seconds = 5
sampling_rate = 512

total_samples = n_seconds * sampling_rate
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = [f'EEG {i}' for i in range(n_channels)]

We create a custom hover tool to make it easier to inspect the values. Each timeseries is built from a `Curve` element, that is given a `label` and that is configured with `subcoordinate_y=True`. They're collected in a list that is given as input to an `Overlay`, displaying it renders a plot with stacked timeseries.

In [ ]:
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = hv.Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel)
    curve.opts(
        subcoordinate_y=True, color="black", line_width=1, tools=[hover],
    )
    channel_curves.append(curve)

eeg = hv.Overlay(channel_curves, kdims="Channel").opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)
eeg

## Minimap data and plot

We compute the *z score* and build an `Image` element configured to display the score on a blue-to-red palette, highlighting where the timeseries diverge the most from their mean value. The `Overlay` we built in the section before has each `Curve` vertically centered around its index value, e.g. `EEG 0` is cerntered around 0. To allow linking between the `Overlay` and the `Image` we are building, we make sure they have the same *yaxis* range by computing `y_positions` appropriately.

In [ ]:
y_positions = range(n_channels)
yticks = [(i , ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = hv.Image((time, y_positions , z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std()*2.5, z_data.std()*2.5)
)
minimap

## Link and dashboard

We link from the `Image` to the `Overlay` using the `RangeToolLink`, set up to link between the two axes and with an initial selection. Our dashboard is finally built by laying out these two elements. Try it out by moving and changing the size of the selection box on the `Image` to explore the timeseries.

In [ ]:
RangeToolLink(
    minimap, eeg, axes=["x", "y"],
    boundsx=(None, 2), boundsy=(None, 4.5)
)

dashboard = (eeg + minimap).opts(merge_tools=False).cols(1)
dashboard